<a href="https://colab.research.google.com/github/rehanhanif3024/Github-test1/blob/test/ATM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# ATM CASH ALERT ML + FASTAPI (UPDATED & SMART)
# ============================================

import joblib
from sklearn.ensemble import RandomForestClassifier

from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import uvicorn
import nest_asyncio
from threading import Thread

# ============================================
# 1. LOAD CSV SAFELY
# ============================================

CSV_PATH = "atm_dummy_data - Sheet1.csv"

data = pd.read_csv(CSV_PATH, sep=None, engine="python")
data.columns = data.columns.str.strip()

print("✅ CSV Loaded")
print(data.head())

# ============================================
# 2. FEATURE ENGINEERING
# ============================================

data["Holiday"] = data["Holiday"].astype(int)

# Target: refill_required based on cash vs avg daily withdrawal
data["refill_required"] = (
    data["Current_Cash"]
    < data["Avg_Daily_Withdrawal"] * data["Holiday"].map({0: 1.0, 1: 1.2})
).astype(int)

FEATURES = [
    "DayOfWeek",
    "Holiday",
    "Current_Cash",
    "Avg_Daily_Withdrawal"
]

X = data[FEATURES]
y = data["refill_required"]

# ============================================
# 3. TRAIN ML MODEL
# ============================================

model = RandomForestClassifier(
    n_estimators=80,
    max_depth=4,
    random_state=42
)

model.fit(X, y)
joblib.dump(model, "atm_model.pkl")

print("✅ Model trained and saved")

# ============================================
# 4. FASTAPI APP
# ============================================

app = FastAPI(title="ATM Cash Refill Alert API")

# Load trained model
model = joblib.load("atm_model.pkl")

# ============================================
# CONFIG
# ============================================
MAX_DAYS_COVERAGE = 3          # Default coverage for recommended refill
CONFIDENCE_THRESHOLD = 0.7     # Only alert if model confidence >= 0.7

# ============================================
# Pydantic Models
# ============================================

class ATMInput(BaseModel):
    atm_id: str
    branch: str
    day_of_week: int
    holiday: int
    current_cash: float
    avg_daily_withdrawal: float

class BatchATMInput(BaseModel):
    atms: List[ATMInput]

# ============================================
# HELPER FUNCTION
# ============================================

def generate_atm_alert(atm: ATMInput):
    df = pd.DataFrame([{
        "DayOfWeek": atm.day_of_week,
        "Holiday": atm.holiday,
        "Current_Cash": atm.current_cash,
        "Avg_Daily_Withdrawal": atm.avg_daily_withdrawal
    }])

    prediction = model.predict(df)[0]
    confidence = model.predict_proba(df)[0][1]

    # Calculate recommended refill amount
    recommended_refill = max(
        0,
        (MAX_DAYS_COVERAGE * atm.avg_daily_withdrawal) - atm.current_cash
    )

    # Build alert message
    if prediction == 1:
        if recommended_refill > 0 and confidence >= CONFIDENCE_THRESHOLD:
            alert_msg = f"REFILL ATM {atm.atm_id} ({atm.branch}) with SAR {round(recommended_refill, 2)}"
        else:
            # Model predicts refill, but enough cash is already available
            alert_msg = (f"ATM {atm.atm_id} ({atm.branch}) cash level OK ")
    else:
        alert_msg = f"ATM {atm.atm_id} ({atm.branch}) cash level OK"

    return {
        "atm_id": atm.atm_id,
        "branch": atm.branch,
        "refill_required": int(prediction),
        "confidence": round(float(confidence), 2),
        "recommended_refill_amount": round(recommended_refill, 2),
        "alert": alert_msg
    }

# ============================================
# ROUTES
# ============================================

@app.get("/")
def root():
    return {"status": "ATM ML API running"}

@app.post("/predict")
def predict_refill(input: ATMInput):
    return generate_atm_alert(input)

@app.post("/predict_batch")
def predict_batch(inputs: BatchATMInput):
    results = [generate_atm_alert(atm) for atm in inputs.atms]
    return results

# ============================================
# RUN FASTAPI (COLAB / LOCAL SAFE)
# ============================================

nest_asyncio.apply()

def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

Thread(target=run_api, daemon=True).start()
print("🚀 FastAPI running on port 8000")


✅ CSV Loaded
   ATM_ID          Branch  DayOfWeek  Holiday  Current_Cash  \
0  ATM001  Riyadh Central          0        0          5000   
1  ATM002     Jeddah Mall          1        0          3000   
2  ATM003     Dammam Souq          2        0          7000   
3  ATM004   Khobar Marina          3        1          2000   
4  ATM005    Mecca Center          4        0          8000   

   Avg_Daily_Withdrawal  
0                  6000  
1                  5000  
2                  6500  
3                  4000  
4                  9000  
✅ Model trained and saved
🚀 FastAPI running on port 8000


In [ ]:
import requests

r = requests.post(
    "http://localhost:8000/predict",
    json={
        "atm_id": "ATM006",
        "branch": "Medina Square",
        "day_of_week": 3,
        "holiday": 0,
        "current_cash": 300000,
        "avg_daily_withdrawal": 4500
    }
)

print(r.json())


INFO:     127.0.0.1:58902 - "POST /predict HTTP/1.1" 200 OK
{'atm_id': 'ATM006', 'branch': 'Medina Square', 'refill_required': 1, 'confidence': 0.76, 'recommended_refill_amount': 0, 'alert': 'ATM ATM006 (Medina Square) cash level OK '}
